# Time-Series Analysis

## Environment set up

Changing the working directory to be able to work with the source-code.

In [56]:
import os
from pathlib import Path

WORKING_DIRECTORY = Path.cwd().parents[0]
os.chdir(WORKING_DIRECTORY)
os.getcwd()

'C:\\python\\personal'

## Imports

In [57]:
from src.read import read_nasa_vibration_files_in_directory
from src.signals import get_resolution
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


## Inputs

In [58]:
DATA_PATH = r'data\1st_test\1st_test'
COLUMN_NAMES = ['channel_1', 'channel_2', 'channel_3', 'channel_4', 
                'channel_5', 'channel_6', 'channel_7', 'channel_8']

SAMPLING_RATE_IN_HERTZ = 20000
MEASUREMENT_DURATION_IN_SECONDS = 1

path = WORKING_DIRECTORY.joinpath(DATA_PATH)


list_of_files =  os.listdir(path)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\python\\personal\\data\\1st_test\\1st_test'

In [ ]:
signal_resolution = get_resolution(sampling_frequency=SAMPLING_RATE_IN_HERTZ)
df_list = read_nasa_vibration_files_in_directory(files_path=path, column_names=COLUMN_NAMES,
                                                 signal_resolution=signal_resolution)

In [ ]:
number_of_measurements = len(df_list)
print(f'number_of_measurements: {number_of_measurements}')

In [ ]:
for channel in COLUMN_NAMES:
    plt.figure(figsize=(12, 3))
    plt.scatter(df_list[-1]['measurement_time_in_seconds'], df_list[-1][channel], label='Last measurement', color='red', alpha=0.5)
    plt.scatter(df_list[0]['measurement_time_in_seconds'], df_list[0][channel], label='First measurement', color='blue', alpha=0.5)
    plt.legend()
    plt.grid()
    plt.title(channel);

In [ ]:
df = pd.concat(df_list, ignore_index=True)

In [ ]:
df.columns

In [ ]:
FAULTY_CHANNELS = ['channel_5', 'channel_6', 'channel_7', 'channel_8']

AGGREGATION_FUNCTIONS = [
    ('mean', 'mean'),
    ('std', 'std'),
    ('max', 'max'),
    ('median', 'median'),
    ('mode', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    ('q1', lambda x: x.quantile(0.25)),
    ('q2', lambda x: x.quantile(0.50)),  # same as median
    ('q3', lambda x: x.quantile(0.75)),
    ('q4', lambda x: x.quantile(1.0))  # same as max
]

aggregated_df = (
    df.groupby('file_name')[FAULTY_CHANNELS]
      .agg(AGGREGATION_FUNCTIONS)
      .reset_index()
)

# Flatten MultiIndex cleanly
aggregated_df.columns = [
    "_".join([str(c) for c in col if c]).rstrip("_")
    for col in aggregated_df.columns.to_flat_index()
]

# Convert file_name into datetime
aggregated_df['date_time'] = pd.to_datetime(
    aggregated_df['file_name'], format="%Y.%m.%d.%H.%M.%S"
)

aggregated_df


In [ ]:
plot_columns

In [ ]:
plot_columns = np.sort(np.array([i for i in aggregated_df.columns if 'channel' in i]))

for plot_column in plot_columns:
    plt.figure(figsize=(12, 2.5))
    plt.scatter(aggregated_df['date_time'], aggregated_df[plot_column])
    plt.title(plot_column)
    plt.grid()